Python's
[subprocess.Popen](https://docs.python.org/3/library/subprocess.html#popen-objects)
class and [signal](https://docs.python.org/3/library/signal.html)
library provide powerful ways to
[communicate with](https://docs.python.org/3/library/subprocess.html#subprocess.Popen.communicate)
and
[send signals to](https://docs.python.org/3/library/subprocess.html#subprocess.Popen.send_signal)
a running subprocess.

We'll look at at using the
[SIGSTOP and SIGCONT signals](https://manpages.debian.org/buster/manpages/signal.7.en.html#Standard_signals)
to pause and resume the execution of a program. As an example,
we'll launch another Python process that runs a `print_time` script.
The `print_time` script takes two arguments, a time delta $n$ and a
total time $t$. The script prints its process ID and the Unix time every
$n$ seconds and completes after $t$ seconds.

The `print_time` script looks like

```python
import os
import sys
import time

if __name__ == "__main__":

    sleep_time = float(sys.argv[-2])
    run_time = float(sys.argv[-1])

    end_time = time.time() + run_time

    while time.time() < end_time:
        print(f"{os.getpid()}: {time.time()}")
        time.sleep(sleep_time)
```

Even though we're running other Python processes in this example, the
concept of using signals applies to any program. The same script
could've been implemented in any language or even with a mix of bash and
Unix programs.

In [1]:
import signal
from subprocess import Popen, PIPE
import time

A few notes about the `Popen` class which launches a running
subprocess. The first argument is a list of strings containing the
program to run and its arguments. This line

```python
Popen(["python", "print_time.py", "1" "5"])
```

Is equivalent to opening your terminal and running

```bash
python print_time.py 1 5
```

The subprocess will begin running as soon as a `Popen` object is
constructed and the `Popen.wait()` method will wait for the subprocess
to terminate.

The `stdout` argument sets where the process's output will be
written. By using `subprocess.PIPE`, it will be written to the
`Popen.stdout` attribute which can be read like a file. The `encoding`
argument sets `stdout` to be interpreted as a stream of text rather
than a stream of bytes.

In the example below, we'll run the `print_time` program for five seconds
with the Unix time printed at one second intervals. Two seconds in, we'll
use the `SIGSTOP` signal to pause the program's execution and then resume
it two seconds later with the `SIGCONT` signal.

In [2]:
p = Popen(["python", "print_time.py", "1", "5"], stdout=PIPE, encoding="utf-8")
time.sleep(2)
p.send_signal(signal.SIGSTOP)
time.sleep(2)
p.send_signal(signal.SIGCONT)
p.wait();  # Semi-colon is so the output of p.wait() is not printed

Note in the output below that the program runs for two seconds,
pauses for a two second gap, then runs for two more seconds.

In [3]:
for line in p.stdout.readlines():
    print(line.strip())

85887: 1584317646.599296
85887: 1584317647.6032739
85887: 1584317650.5725708
85887: 1584317651.574015


For a more involved example, we'll launch three processes
running `print_time` and stop all of them immediately. We'll then
use `SIGSTOP` and `SIGCONT` to allow each of them to run one at a
time in round-robin fashion until they've all exited. This functions
like a time-slicing mechanism in which each program gets equal
execution time and none of the programs run concurrently.

In [4]:
processes = [
    Popen(["python", "print_time.py", "1", "8"], stdout=PIPE, encoding="utf-8")
    for _ in range(3)
]
for process in processes:
    process.send_signal(signal.SIGSTOP)

The `Popen.poll()` method returns `None` if a process is still
running. We'll use this to check whether any of the processes
are still running and, if so, continue to run each process
one-by-one.

In [5]:
while any(process.poll() is None for process in processes):
    for process in processes:
        process.send_signal(signal.SIGCONT)
        time.sleep(2)
        process.send_signal(signal.SIGSTOP)

To see how the processes ran in a non-overlapping fashion, we can
combine the output of each process into one list and then sort
the list by the printed Unix time value. This will reconstruct
which process was running at each timestamp.

In [6]:
lines = [
    line.strip()
    for process in processes
    for line in process.stdout.readlines()
]
lines.sort(key=lambda line: float(line.split()[-1]))

In the output below, you can see each process run for two seconds before
switching to a different process.

In [7]:
print("\n".join(lines))

85888: 1584317652.730904
85888: 1584317653.731411
85889: 1584317654.723969
85889: 1584317655.7270458
85890: 1584317656.732857
85890: 1584317657.7348201
85888: 1584317658.693002
85888: 1584317659.693248
85889: 1584317660.693377
85889: 1584317661.698638
85890: 1584317662.693693
85890: 1584317663.694156
85890: 1584317668.701111


## Parting thoughts

The example above does, in a very rough way, what
an [ARINC 653](https://en.wikipedia.org/wiki/ARINC_653)
operating system does in partitioning applications by time.

A fun exercise would be to write a program that takes a config file
containing programs with a time-slice allocation and coordinates
their execution. It would be interesting to come up with techniques
for precise timing control and measuring how well the OS and Python
interpreter are able to switch between the running programs.

My idea for this came while watching
[this command-line environment](https://missing.csail.mit.edu/2020/command-line/)
lecture which does a great job giving an overview of signals and a
number of other things. The entire
[Missing Semester lecture series](https://missing.csail.mit.edu/) is
really fantastic and something that I wish I'd taken a college course on. I
found the lectures on
[editors](https://missing.csail.mit.edu/2020/editors/),
[data wrangling](https://missing.csail.mit.edu/2020/data-wrangling/),
and [version control](https://missing.csail.mit.edu/2020/version-control/)
to be particularly useful.